https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.pdf

In [26]:
import pandas as pd
import requests

In [27]:
#Read the VF file into a dataframe
dfV = pd.read_csv('./data/NC-County-Wake-VF.csv')

In [28]:
#Create a function that will invoke the CENSUS API to return coordinates for an address
def getCoord(address):
    #Form the request
    censusURL = 'http://geocoding.geo.census.gov/geocoder/locations/onelineaddress'
    params = {'benchmark':'9',#'returntype':'locations',
          'address':address,
          'format':'json',
         } 
    #Send the request
    response = requests.get(censusURL, params)
    #Read in the response as a JSON encoded object
    jsonObj = response.json()
    addresses = jsonObj['result']['addressMatches']
    #If no valid addresses are returned, send coordinates of (-1,-1)
    if len(addresses)== 0:
        print("X",end='')
        return((-1,-1))
    #Otherswise return the coordinate
    else:
        print(".",end='')
        coords = addresses[0]['coordinates']
        lat = float(coords['y'])
        lng = float(coords['x'])
        #Log results
        with open("Log.csv",'a') as fObj: 
            fObj.write("{},{},{}\n".format(address,lat,lng))
        #Return the coordinate pair for the address
        return(lat, lng)

In [29]:
#Create an address column
dfV['address'] = (dfV['address1__registered_address'] + " " 
                  + dfV['city__registered_address'] + " " 
                  + dfV['state__registered_address'] + " "
                  + dfV['zip__registered_address'].astype(str))

In [ ]:
#Convert address to coordinates
dfV['coords'] = dfV['address'].apply(lambda x: getCoord(x))

X.X...............X..X..............................X..X........X.X......X.XX.......XX...........X...X..X.......................X.XX.......X...X...X.........X.....X...X.X..X.....X.......X.X.........X...X.X.X.........X.......XX....X...........X...XX..X......X......X........X......X...X......X.................X.........X......X..........X.................X.....X......X..................X.......X..X......XX...X......XX.....X....X.X...XX.X.......X....X...........X..................................X....X...X.......X.XX...X........X..........X..X.X......................X...................X.X...........XX..X.......X.....X.........X.....X...X...X...X....X........XX.....X......X.....X..X.......X............X........X...X........X...................X....................X..X.X.X.......X.....X..X...XX...............XXXX..X.X...........X.........XXX....XX.XX.X....XX.XX............X...XXX.........X..XX............X..X.....X.X.......X...............XX....X.XX........X.......XX...XX.X....XX.....X....

..X....X.......X..XX.X.......XXX.XXX....XXX...X.XX.....X..............X....X.XX....X.XX.XX.XX.......X...X....XXX..X.X..X..XXX.X.....X..X..X....X.......X.X.X..........X...............X....X....X..........X.X....X.........X.XX.X.XX........................X..X.X........XXX.X.XXXXX.XX................X..X.X.....................X.................XX...........X.............X.....X............X..XX...X..XX.......X..X...X......X...X.....X..................X..................X..XX.......................X...X........XX.X....X..............XXX..X..X...X.....X..X.......X......XX....................XX............X..X...........X....X...XX.............X......X..................X.......X....XX.XX.............X.....X...X.X...XX.......X...XX...........X....

In [ ]:
#Write the results
dfV.to_csv('./data/NC-County-Wake-VF_coords.csv',index=False)